In [2]:
!pip install triton

In [1]:
import torch
import triton
import triton.language as tl

In [7]:
@triton.jit
def stridemultiply(inputptr,outputptr,stride,n,block_size: tl.constexpr):
    blockid=tl.program_id(0)
    block_start= blockid * block_size 
    offsets=block_start+tl.arange(0,block_size)
    strided_offsets=offsets*stride
    mask=offsets<n
    x=tl.load(inputptr+strided_offsets,mask=mask)
    result=x*2
    tl.store(outputptr+strided_offsets,result,mask=mask)

In [9]:
def test():
    data = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], device='cuda', dtype=torch.float32)
    n=len(data)

    output2 = data.clone()
    BLOCK_SIZE = 256  
    stride=3
    num_blocks = triton.cdiv(n, BLOCK_SIZE)  
    stridemultiply[(num_blocks,)](data, output2,stride, n, BLOCK_SIZE)
    print(f"New way result: {output2}")

if __name__ == "__main__":
    test()

New way result: tensor([ 2.,  2.,  3.,  8.,  5.,  6., 14.,  8.,  9., 20.], device='cuda:0')
